In [2]:
import sqlite3

In [24]:

import csv
import random
import math


con= sqlite3.connect('tracks.db')
cur= con.cursor()
# droping tables
cur.execute('DROP TABLE IF EXISTS Artist')
cur.execute('DROP TABLE IF EXISTS Album')
cur.execute('DROP TABLE IF EXISTS Track')

# creating tables
cur.execute('''CREATE TABLE Artist (
    id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL UNIQUE, 
    artist_name TEXT)
 ''')

cur.execute('''CREATE TABLE Album (
    id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL UNIQUE, 
    album_name TEXT,
    artist_id INTEGER
    )''')

cur.execute('''CREATE TABLE Track (
    id INTEGER PRIMARY KEY AUTOINCREMENT NOT NULL UNIQUE, 
    track_name TEXT,
    album_id INTEGER,  duration TEXT, rating REAL)
    ''')



f= open('songs.csv')
reader= csv.DictReader(f)

for row in reader:
    duration= row['duration_ms']
    duration=float(duration)*0.00001667 # millieseconnds to mins change.
    mins= math.floor(duration)
    duration_str= f'{mins} mins {math.floor(60*(duration- mins))} secs'
    artist= row['artist_name']
    track= row['track_name']
    album= row['album_name']
    rating= round(random.uniform(1,5),1)
    cur.execute('INSERT OR IGNORE INTO Artist(artist_name) VALUES(?)', (artist,))
    con.commit()
    cur.execute('SELECT id from Artist where artist_name=?', (artist,))
    artist_id= cur.fetchone()[0]
    cur.execute('INSERT OR IGNORE INTO Album(album_name, artist_id) VALUES(?, ?)', (album,artist_id))
    con.commit()
    cur.execute('SELECT id from Album where album_name=? and artist_id=?', (album,artist_id))
    album_id= cur.fetchone()[0]
    cur.execute('INSERT OR IGNORE INTO Track(track_name,album_id ,  duration , rating ) VALUES(?, ?,?, ?)', (track,album_id,duration_str,rating))
    con.commit()

print('process complete')
con.close()

process complete


In [21]:
# Grabing data from table

track_name= input('Enter name of the song: ')
con= sqlite3.connect('tracks.db')
cur= con.cursor()

sql='''SELECT Track.track_name,  Album.album_name, Artist.artist_name,Track.duration, Track.rating from 
Track join Album join Artist on Track.album_id== Album.id and Album.artist_id= Artist.id where track_name like ?'''

cur.execute(sql, ('%'+track_name+'%',))
data= cur.fetchall()
if len(data)>0:
    print('{:30}{:30}{:30}{:15}{:7}'.format('Track Name','Album Name', 'Artist', 'Duration', 'Rating'))
    for row in data:
        if len(row[0])>30:
            d1= row[0][:25]+'...'
        else:
            d1= row[0]
        if len(row[1])>30:
            d2= row[1][:25]+'...' 
        else:
            d2= row[1]
            
        
        print('{:30}{:30}{:30}{:15}{:>7.2f}'.format(d1,d2,row[2], row[3], row[4]))
else:
    print('No Matches found')
        
con.close()

Enter name of the song: christmas
Track Name                    Album Name                    Artist                        Duration       Rating 
White Christmas               White Christmas               Bing Crosby                   3 mins 3 secs     3.90
The Christmas Waltz           A Swingin' Christmas Feat...  Tony Bennett                  3 mins 19 secs    4.60
I Don't Want to Go Home f...  I Don't Want to Go Home f...  Independent Counsel of Funk   3 mins 23 secs    1.90
Baby! It's Christmas          Baby! It's Christmas          Jessie James Decker           3 mins 7 secs     5.00
All I Want for Christmas ...  Merry Christmas               Mariah Carey                  4 mins 1 secs     2.60
The Christmas Waltz           A Very She & Him Christmas    She & Him                     2 mins 37 secs    2.00
Last Christmas (Glee Cast...  Glee: The Music, The Chri...  Glee Cast                     3 mins 36 secs    3.50
Lonely This Christmas         Let's Have A Party - The ...  Mu